In [236]:
# Tool Setup

%load_ext tensorboard

import pathlib, os, sys, operator, re, datetime

from functools import reduce
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import Model
import tensorflow_datasets as tfds
import h5py
import pandas as pd

# Use seaborn for pairplot.
# pip install -q seaborn

import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

print(tf.__version__)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
2.19.0


# 2024 Dataset

In [252]:

url2 = 'p:\Mitchell\FFNeuralNetwork\statsTesting.csv'

raw_dataset2024 = pd.read_csv(url2, na_values='?', comment='\t', skipinitialspace=True)

dataset2024 = raw_dataset2024.copy()

dataset2024.tail()

,Age,Previous Rank,FantPos,Games Played,Games Started,Cmp,Att,Yds,TD,Int,Rush Att,Rush Yds,Rush TD,Tgt,Rec,Rec Yds,Rec TD,Total TD,PPR,Rank
625,25,0,0,10,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-2.0,626
626,27,474,1,9,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-2.0,627
627,25,434,2,6,0,2.0,2.0,8.0,0.0,0.0,7.0,-4.0,0.0,0.0,0.0,0.0,0.0,0,-2.1,628
628,25,295,1,3,0,0.0,0.0,0.0,0.0,0.0,2.0,-4.0,0.0,1.0,0.0,0.0,0.0,0,-2.4,629
629,24,0,1,12,0,0.0,0.0,0.0,0.0,0.0,1.0,-5.0,0.0,3.0,1.0,8.0,0.0,0,-2.7,630


In [253]:
dataset2024.isna().sum()
dataset2024 = dataset2024.dropna(axis = 0, subset = 'Previous Rank')
dataset2024 = dataset2024.fillna(0)
testSet2024 = dataset2024.copy()
# datasetAns = datasetAns.fillna(0)

# dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')

# datasetAns.tail()
dataset2024.tail()

,Age,Previous Rank,FantPos,Games Played,Games Started,Cmp,Att,Yds,TD,Int,Rush Att,Rush Yds,Rush TD,Tgt,Rec,Rec Yds,Rec TD,Total TD,PPR,Rank
625,25,0,0,10,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-2.0,626
626,27,474,1,9,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-2.0,627
627,25,434,2,6,0,2.0,2.0,8.0,0.0,0.0,7.0,-4.0,0.0,0.0,0.0,0.0,0.0,0,-2.1,628
628,25,295,1,3,0,0.0,0.0,0.0,0.0,0.0,2.0,-4.0,0.0,1.0,0.0,0.0,0.0,0,-2.4,629
629,24,0,1,12,0,0.0,0.0,0.0,0.0,0.0,1.0,-5.0,0.0,3.0,1.0,8.0,0.0,0,-2.7,630


In [254]:
y = testSet2024[['Rank']]
dataset = dataset2024.drop('Rank', axis = 1)
X = dataset2024[['Previous Rank', 'PPR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.2, random_state=42)
#
X.tail()
dataset.tail()

,Age,Previous Rank,FantPos,Games Played,Games Started,Cmp,Att,Yds,TD,Int,Rush Att,Rush Yds,Rush TD,Tgt,Rec,Rec Yds,Rec TD,Total TD,PPR
625,25,0,0,10,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-2.0
626,27,474,1,9,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-2.0
627,25,434,2,6,0,2.0,2.0,8.0,0.0,0.0,7.0,-4.0,0.0,0.0,0.0,0.0,0.0,0,-2.1
628,25,295,1,3,0,0.0,0.0,0.0,0.0,0.0,2.0,-4.0,0.0,1.0,0.0,0.0,0.0,0,-2.4
629,24,0,1,12,0,0.0,0.0,0.0,0.0,0.0,1.0,-5.0,0.0,3.0,1.0,8.0,0.0,0,-2.7


# Large Dataset

In [ ]:
url = 'p:\Mitchell\FFNeuralNetwork\statsTraining.csv'
# ans = 'p:\Mitchell\FFNeuralNetwork\statsAnswerV2.csv'

raw_dataset = pd.read_csv(url, na_values='?', comment='\t', skipinitialspace=True)
# raw_answer = pd.read_csv(ans, na_values='?', comment='\t', skipinitialspace=True)

dataset = raw_dataset.copy()
# datasetAns = raw_answer.copy()
# dataset.tail()

In [238]:
dataset.isna().sum()
dataset = dataset.dropna(axis = 0, subset = 'Previous Rank')
dataset = dataset.fillna(0)
testSet = dataset.copy()
# datasetAns = datasetAns.fillna(0)

# dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')

# datasetAns.tail()
dataset.tail()

,Age,Previous Rank,FantPos,Games Played,Games Started,Cmp,Att,Pass Yds,Pass TD,Int,Rush Att,Rush Yds,Rush TD,Tgt,Rec,Rec Yds,Rec TD,Total TD,PPR,Rank
3240,24,526.0,1,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,627
3241,27,368.0,2,7,0,0.0,0.0,0.0,0.0,0.0,13.0,-13.0,0.0,0.0,0.0,0.0,0.0,0,-1.3,628
3242,24,205.0,0,4,1,0.0,0.0,0.0,0.0,0.0,14.0,16.0,0.0,6.0,5.0,14.0,0.0,0,4.0,629
3244,27,484.0,1,10,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0,-2.0,631
3245,29,455.0,1,17,0,0.0,0.0,0.0,0.0,0.0,1.0,-2.0,0.0,2.0,0.0,0.0,0.0,0,-2.2,632


In [239]:
y = testSet[['Rank']]
dataset = dataset.drop('Rank', axis = 1)
X = dataset[['Previous Rank', 'PPR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.2, random_state=42)
#
X.tail()
dataset.tail()

,Age,Previous Rank,FantPos,Games Played,Games Started,Cmp,Att,Pass Yds,Pass TD,Int,Rush Att,Rush Yds,Rush TD,Tgt,Rec,Rec Yds,Rec TD,Total TD,PPR
3240,24,526.0,1,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3241,27,368.0,2,7,0,0.0,0.0,0.0,0.0,0.0,13.0,-13.0,0.0,0.0,0.0,0.0,0.0,0,-1.3
3242,24,205.0,0,4,1,0.0,0.0,0.0,0.0,0.0,14.0,16.0,0.0,6.0,5.0,14.0,0.0,0,4.0
3244,27,484.0,1,10,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0,-2.0
3245,29,455.0,1,17,0,0.0,0.0,0.0,0.0,0.0,1.0,-2.0,0.0,2.0,0.0,0.0,0.0,0,-2.2


In [255]:
model = LinearRegression()

model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [256]:
y_pred = model.predict(X_test)

In [242]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_test['Previous Rank'], X_test['PPR'],
           y_test, color='blue', label='Actual Data')

        #    X_test['FantPos'], X_test['Games Played'], X_test['Games Started'], X_test['Cmp'], X_test['Att'], X_test['Pass Yds'], X_test['Pass TD'], X_test['Int'],
        #    X_test['Rush Att'], X_test['Rush Yds'], X_test['Rush TD'], X_test['Tgt'], X_test['Rec'], X_test['Rec Yds'], X_test['Rec TD'], X_test['PPR']

x1_range = np.linspace(X_test['Previous Rank'].min(), X_test['Previous Rank'].max(), 100)
# x2_range = np.linspace(X_test['Age'].min(), X_test['Age'].max(), 100)
# x3_range = np.linspace(X_test['FantPos'].min(), X_test['FantPos'].max(), 100)
# x4_range = np.linspace(X_test['Games Played'].min(), X_test['Games Played'].max(), 100)
# x5_range = np.linspace(X_test['Games Started'].min(), X_test['Games Started'].max(), 100)
# x6_range = np.linspace(X_test['Cmp'].min(), X_test['Cmp'].max(), 100)
# x7_range = np.linspace(X_test['Att'].min(), X_test['Att'].max(), 100)
# x8_range = np.linspace(X_test['Age'].min(), X_test['Age'].max(), 100)
# x9_range = np.linspace(X_test['Pass Yds'].min(), X_test['Pass Yds'].max(), 100)
# x10_range = np.linspace(X_test['Pass TD'].min(), X_test['Pass TD'].max(), 100)
# x11_range = np.linspace(X_test['Int'].min(), X_test['Int'].max(), 100)
# x12_range = np.linspace(X_test['Rush Att'].min(), X_test['Rush Att'].max(), 100)
# x13_range = np.linspace(X_test['Rush Yds'].min(), X_test['Rush Yds'].max(), 100)
# x14_range = np.linspace(X_test['Rush TD'].min(), X_test['Rush TD'].max(), 100)
# x15_range = np.linspace(X_test['Tgt'].min(), X_test['Tgt'].max(), 100)
# x16_range = np.linspace(X_test['Rec'].min(), X_test['Rec'].max(), 100)
# x17_range = np.linspace(X_test['Rec Yds'].min(), X_test['Rec Yds'].max(), 100)
# x18_range = np.linspace(X_test['Rec TD'].min(), X_test['Rec TD'].max(), 100)
x19_range = np.linspace(X_test['PPR'].min(), X_test['PPR'].max(), 100)

x1, x2= np.meshgrid(x1_range, x19_range)
z = model.predict(np.c_[x1.ravel(), x2.ravel()]).reshape(x1.shape)

ax.plot_surface(x1, x2, z, color='red', alpha=0.5, rstride=100, cstride=100)

ax.set_xlabel('Previous Rank')
ax.set_ylabel('PPR')
ax.set_zlabel('Rank')
ax.set_title('Multiple Linear Regression Best Fit Line (3D)')

plt.show()

C:\Users\mdriscoll\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


KeyboardInterrupt: 

In [266]:
ex = dataset.iloc[4]
# ex = ex.drop('Rank')

# result = model.predict([ex])      # Multi Variable
result = model.predict([[150, 150]])        # Double Variable
print("Model Predicts Rank of", result)
ex.tail()

Model Predicts Rank of [[178.779]]


C:\Users\mdriscoll\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Rec           0.0
Rec Yds       0.0
Rec TD        0.0
Total TD      3.0
PPR         365.8
Name: 4, dtype: float64